# Data preparation

### X will contain all resized images of cars


In [1]:
import numpy as np
import os
import glob

In [2]:
IMAGE_SIZE = 224

In [3]:
img_dir = "../images"
data_path = os.path.join(img_dir, "*png")
files = glob.glob(data_path)
files.sort()

In [4]:
import cv2

In [5]:
X = list()
for file in files:
    img = cv2.imread(file)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    X.append(np.array(img))

### Y variable contains all resized bounding box annotations (labels)

In [6]:
from lxml import etree

In [7]:
def resize_annotation(annotation_file):
    tree = etree.parse(annotation_file)
    for dim in tree.xpath("size"):
        width = int(dim.xpath("width")[0].text)
        height = int(dim.xpath("height")[0].text)
    for dim in tree.xpath("object/bndbox"):
        xmin = int(dim.xpath("xmin")[0].text) / (width / IMAGE_SIZE)
        ymin = int(dim.xpath("ymin")[0].text) / (height / IMAGE_SIZE)
        xmax = int(dim.xpath("xmax")[0].text) / (width / IMAGE_SIZE)
        ymax = int(dim.xpath("ymax")[0].text) / (height / IMAGE_SIZE)
    return [int(xmax), int(ymax), int(xmin), int(ymin)]

In [8]:
annotation_path = "../annotations/"
annotation_files = [annotation_path + f for f in sorted(os.listdir(annotation_path))]

In [9]:
y = list()
for annotation_file in annotation_files:
    y.append(resize_annotation(annotation_file))

## Example of resized annotation file

In [ ]:
resize_annotation(f"{annotation_path}" + "Cars0.xml")

In [ ]:
np.array(X).shape

In [ ]:
np.array(y).shape

## Car images examples

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(10, 20), dpi=200)
for number in range(15):
    plt.subplot(10, 5, number + 1)
    plt.axis("off")
    plt.imshow(X[number])

## Car image examples with annotations

In [ ]:
plt.figure(figsize=(10, 20), dpi=200)
for number in range(15):
    plt.subplot(10, 5, number + 1)
    plt.axis("off")

    image = cv2.rectangle(
        X[number],
        (y[number][0], y[number][1]),
        (y[number][2], y[number][3]),
        (0, 0, 255),
    )
    plt.imshow(image)

## Data transformation to Numpy array

In [ ]:
X = np.array(X)
y = np.array(y)

## Data normalization

In [ ]:
X = X / 255
y = y / 255

# Train | Validation | Test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_val_test, y_val_test, test_size=0.5, random_state=42
)

# Convolutionnal Neural Network

## Model Imports

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input

## Creating the model

In [ ]:
model = Sequential()
model.add(
    InceptionV3(
        input_tensor=Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
        weights="imagenet",
        include_top=True,
    )
)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(8, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

model.layers[-9].trainable = False

In [ ]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

## Early stopping to avoid unnecessary training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=50)

## Model training

In [ ]:
training_history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=1000,
    batch_size=64,
    verbose=1,
    callbacks=[early_stop],
)

## Save the model for future use

In [ ]:
model_name = "InceptionV3_batch_64"

In [ ]:
model.save(f"../saved_models/car_plate_detection_{model_name}.h5", overwrite=True)

## Model evaluation
### Final results

In [ ]:
loss_score, accuracy_score = model.evaluate(X_test, y_test, verbose=0)

print(f"Loss: {(loss_score * 100):.2f} %")
print(f"Accuracy: {(accuracy_score * 100):.2f} %")

In [ ]:
accuracy = training_history.history["accuracy"]
val_accuracy = training_history.history["val_accuracy"]
epochs = range(len(accuracy))

plt.figure(figsize=(20, 10), dpi=200)
plt.plot(epochs, accuracy, "blue", label="Accuracy")
plt.plot(epochs, val_accuracy, "red", label="Val accuracy")
plt.title("Scores")
plt.legend()

plt.savefig(f"../saved_images/accuracy_scores_{model_name}.jpg", bbox_inches="tight")
plt.show()

# Model predictions

In [ ]:
predictions = model.predict(X_test) * 255

In [ ]:
plt.figure(figsize=(10, 20), dpi=200)
for number in range(15):
    plt.subplot(10, 5, number + 1)
    plt.axis("off")
    prediction = predictions[number]
    image = cv2.rectangle(
        X_test[number],
        (int(prediction[0]), int(prediction[1])),
        (int(prediction[2]), int(prediction[3])),
        (0, 255, 0),
    )
    plt.imshow(image)

plt.savefig(f"../saved_images/predictions_{model_name}.jpg", bbox_inches="tight")